<a href="https://colab.research.google.com/github/jahnaviakurathi/machinelearning/blob/main/sss3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

!pip install tensorflow

# Import necessary libraries

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Step 1: Frame Extraction
def extract_frames(video_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    cap.release()
    print(f"Extracted {frame_count} frames to {output_folder}.")

# Step 2: CNN Feature Extraction
def extract_cnn_features(frame_folder):
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    features = []

    for frame_file in sorted(os.listdir(frame_folder)):
        frame_path = os.path.join(frame_folder, frame_file)
        img = cv2.imread(frame_path)
        img = cv2.resize(img, (224, 224))
        img = preprocess_input(np.expand_dims(img, axis=0))
        feature = model.predict(img)
        features.append(feature.flatten())

    features = np.array(features)
    print(f"Extracted CNN features of shape: {features.shape}.")
    return features

# Step 3: Reshape Frame Features for RNN Input
def reshape_features_for_rnn(features, sequence_length):
    num_sequences = len(features) - sequence_length + 1
    rnn_input = np.array([features[i:i + sequence_length] for i in range(num_sequences)])
    return rnn_input

# Step 4: RNN Model Definition
def create_rnn_model(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        LSTM(32),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model


# Step 5: Violence Detection Pipeline
def detect_violence(rnn_model, rnn_input, threshold=0.5):
    predictions = rnn_model.predict(rnn_input)
    flagged_frames = []

    for i, prob in enumerate(predictions):
        # Ensure 'prob' is correctly interpreted as a scalar value
        prob = prob[0] if isinstance(prob, (np.ndarray, list)) else prob

        if prob > threshold:
            flagged_frames.append(i)
            print(f"Violence detected in sequence {i} with probability {prob:.2f}.")

    return flagged_frames



# Main Function
if __name__ == "__main__":
    video_path = "/content/18.mp4"  # video file
    frame_folder = "frames"
    sequence_length = 10  # Number of frames per sequence

    # Step 1: Frame Extraction
    extract_frames(video_path, frame_folder)

    # Step 2: CNN Feature Extraction
    features = extract_cnn_features(frame_folder)

    # Step 3: Reshape Features for RNN Input
    rnn_input = reshape_features_for_rnn(features, sequence_length)
    print(f"RNN input shape: {rnn_input.shape}.")

    # Step 4: Train-Test Split (Mock labels for demonstration purposes)
    labels = np.random.randint(0, 2, size=(len(rnn_input),))  # Replace with real labels
    X_train, X_test, y_train, y_test = train_test_split(rnn_input, labels, test_size=0.2, random_state=42)

    # Define RNN model
    rnn_model = create_rnn_model(rnn_input.shape[1:])
    print(rnn_model.summary())

    # Train RNN model
    rnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=8)


    # Step 5: Violence Detection
flagged_frames = detect_violence(rnn_model, X_test, threshold=0.5)

if flagged_frames:
    print(f"Flagged sequences: {flagged_frames}")
else:
    print("No violent sequences detected.")






Extracted 192 frames to frames.
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/s

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_14 (LSTM)                       │ (None, 10, 64)              │         540,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_15 (LSTM)                       │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 553,377 (2.11 MB)

 Trainable params: 553,377 (2.11 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - accuracy: 0.5565 - loss: 0.7227 - val_accuracy: 0.5135 - val_loss: 0.6997
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.4195 - loss: 0.7108 - val_accuracy: 0.5135 - val_loss: 0.6937
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.5575 - loss: 0.6883 - val_accuracy: 0.5135 - val_loss: 0.6918
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5935 - loss: 0.6761 - val_accuracy: 0.5135 - val_loss: 0.6970
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.4264 - loss: 0.7090 - val_accuracy: 0.5135 - val_loss: 0.6925
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.5169 - loss: 0.6923 - val_accuracy: 0.5135 - val_loss: 0.7022
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.4727 - loss: 0.7168 - val_accuracy: 0.5135 - val_loss: 0.6956
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5785 - loss: 0.6909 - val_accuracy: 0.513